<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/ro_whisper_tiny_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ryr4klwz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ryr4klwz
  Resolved https://github.com/huggingface/transformers to commit 27b752bcf16708dc062a1d69325a07e4560aa75a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8049178 sha256=1bc69631b329058939cc9ec4b8f6014ae755caaf64d2b5afff532c52afd8bd85
  Stored in directory: /tmp/pip-ephem-wheel-cache-8s2q06b6/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.1 M

In [3]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm


In [21]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "ro", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [22]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1383
})

In [23]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [24]:
model = whisper.load_model("tiny")

In [25]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [26]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [27]:
text

,0
0,vorbim pe drept cuvânt despre noua zeelandă ca...
1,raportul comitetului de supervizare e foarte n...
2,comisia a anunțat publicarea evaluării impactu...
3,este preferabil să investim în construcia de c...
4,această măsură ar urma și ar permite să realiz...
...,...
1378,aplicarea acestor lucruri care sunt deja în le...
1379,singurul argument prezentat de chișinău în fav...
1380,în ceea ce privește raportul privind protecția...
1381,s a încălcat astfel articolul douăzeci din con...


In [28]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [29]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,vorbim pe drept cuvânt despre noua zeelandă ca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0079345703125, 0.000152587890625, -0.005004...",16000
1,female,raportul comitetului de supervizare e foarte n...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00067138671875, -0.001556396484375, -0.0028...",16000
2,female,comisia a anunțat publicarea evaluării impactu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007659912109375, -0.000885009765625, -0.009...",16000
3,female,este preferabil să investim în construcia de c...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00140380859375, -0.0010986328125, 0.0, -0....",16000
4,male,această măsură ar urma și ar permite să realiz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002166748046875, 0.001708984375, 0.00146484...",16000
...,...,...,...,...,...
1378,female,aplicarea acestor lucruri care sunt deja în le...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0250244140625, -0.01055908203125, -0.029266...",16000
1379,male,singurul argument prezentat de chișinău în fav...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002960205078125, 0.002166748046875, 0.00234...",16000
1380,male,în ceea ce privește raportul privind protecția...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00115966796875, -0.004852294921875, -0.001...",16000
1381,male,s a încălcat astfel articolul douăzeci din con...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005859375, -0.002197265625, -0.017059326171...",16000


In [30]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,vorbim pe drept cuvânt despre noua zeelandă ca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0079345703125, 0.000152587890625, -0.005004...",16000
4,male,această măsură ar urma și ar permite să realiz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002166748046875, 0.001708984375, 0.00146484...",16000
5,male,doamnă președinte dacă vrem să realizăm cu ade...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.000762939453125, 9.15527...",16000
8,male,cât despre conținutul modificărilor soluțiile ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00177001953125, 0.0, 0.000335693359375, 0....",16000
9,male,rezoluia pe care am votat o astăzi transmite u...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00201416015625, 0.0025634765625, 0.00210571...",16000
...,...,...,...,...,...
1377,male,nu în ultimul rând consider că parteneriatul s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00616455078125, -0.006622314453125, -0.006...",16000
1379,male,singurul argument prezentat de chișinău în fav...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002960205078125, 0.002166748046875, 0.00234...",16000
1380,male,în ceea ce privește raportul privind protecția...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00115966796875, -0.004852294921875, -0.001...",16000
1381,male,s a încălcat astfel articolul douăzeci din con...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005859375, -0.002197265625, -0.017059326171...",16000


In [31]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,raportul comitetului de supervizare e foarte n...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00067138671875, -0.001556396484375, -0.0028...",16000
2,female,comisia a anunțat publicarea evaluării impactu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007659912109375, -0.000885009765625, -0.009...",16000
3,female,este preferabil să investim în construcia de c...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00140380859375, -0.0010986328125, 0.0, -0....",16000
6,female,propaganda rusă iarăși trebuie contracarată se...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, 0.00439453125, 0.00531005...",16000
7,female,ameliorarea formării agenilor de control un ni...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00152587890625, 0.003265380859375, 0.001312...",16000
...,...,...,...,...,...
1370,female,apreciem propunerea comisie de a combina aciun...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002105712890625, 0.00189208984375, 0.00631...",16000
1372,female,consecinele crizei actuale au un impact negati...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001983642578125, -0.001312255859375, -0.00...",16000
1374,female,de aceea regulamentul prevede obligaia pentru ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0006103515625, -0.00018310546875, 0.0020751...",16000
1376,female,și dumneavoastră domnule comisar avramopoulos ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.070220947265625, -0.062896728515625, -0.07...",16000


In [33]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="ro")
    return result["text"]

In [34]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [35]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.5833333333333334
Processed 2 files, WER: 0.6190476190476191
Processed 3 files, WER: 0.8387096774193549
Processed 4 files, WER: 0.6333333333333333
Processed 5 files, WER: 0.5862068965517241
Processed 6 files, WER: 0.76
Processed 7 files, WER: 0.6666666666666666
Processed 8 files, WER: 0.8181818181818182
Processed 9 files, WER: 0.76
Processed 10 files, WER: 0.8666666666666667
Processed 11 files, WER: 0.8823529411764706
Processed 12 files, WER: 0.42857142857142855
Processed 13 files, WER: 1.0
Processed 14 files, WER: 0.65
Processed 15 files, WER: 0.45714285714285713
Processed 16 files, WER: 0.6956521739130435
Processed 17 files, WER: 1.0625
Processed 18 files, WER: 0.7391304347826086
Processed 19 files, WER: 0.6666666666666666
Processed 20 files, WER: 0.42857142857142855
Processed 21 files, WER: 0.8888888888888888
Processed 22 files, WER: 0.7777777777777778
Processed 23 files, WER: 0.4838709677419355
Processed 24 files, WER: 0.6285714285714286
Processed 25 files,

In [36]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.75
Processed 2 files, WER: 0.8
Processed 3 files, WER: 0.391304347826087
Processed 4 files, WER: 1.0
Processed 5 files, WER: 0.625
Processed 6 files, WER: 0.6578947368421053
Processed 7 files, WER: 0.8620689655172413
Processed 8 files, WER: 0.7647058823529411
Processed 9 files, WER: 0.6521739130434783
Processed 10 files, WER: 0.7884615384615384
Processed 11 files, WER: 0.6470588235294118
Processed 12 files, WER: 0.9444444444444444
Processed 13 files, WER: 0.8793103448275862
Processed 14 files, WER: 0.7560975609756098
Processed 15 files, WER: 0.8333333333333334
Processed 16 files, WER: 1.4
Processed 17 files, WER: 0.9487179487179487
Processed 18 files, WER: 0.9
Processed 19 files, WER: 0.4444444444444444
Processed 20 files, WER: 1.0
Processed 21 files, WER: 0.7777777777777778
Processed 22 files, WER: 0.2857142857142857
Processed 23 files, WER: 0.6428571428571429
Processed 24 files, WER: 0.7058823529411765
Processed 25 files, WER: 0.782608695652174
Processed 26 